In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
# getting the current day for url
today = datetime.now()
day_of_month = today.day
month = today.month
year = today.year

In [3]:
URL = f"https://climate.weather.gc.ca/historical_data/search_historic_data_stations_e.html?searchType=stnProv&timeframe=1&lstProvince=&optLimit=yearRange&StartYear=1840&EndYear=2022&Year={year}&Month={month}&Day={day_of_month}&selRowPerPage=100"
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"}
municipal_urls = {}
chosen_provinces = ['BC']

In [4]:
r = requests.get(URL, headers=headers)
soup = BeautifulSoup(r.content)

In [5]:
# getting next button
def is_last_page(soup, page):
    button_row = soup.find('div', {'class': 'pull-left'}).findAll('li')
    if button_row[-1].find('a')['href'] == '':
        last_button_num = int(button_row[-2].find('a').text)
        return (last_button_num - 1) * 100 + 1 == (page - 1) * 100 + 1
    else:
        return False

In [6]:
def get_page_links(soup):
    for municipality in soup.find('div', {'class': 'historical-data-results'}).findAll('form'):
        hlyRange = municipality.find('input', {'name': 'hlyRange'})['value']
        dlyRange = municipality.find('input', {'name': 'dlyRange'})['value']
        mlyRange = municipality.find('input', {'name': 'mlyRange'})['value']
        station_id = municipality.find('input', {'name': 'StationID'})['value']
        prov = municipality.find('input', {'name': 'Prov'})['value']
        mun_name = municipality.find('div', {'class': 'col-lg-3'}).text
        search_year = municipality.find('option', {'selected': ''})['value']
        search_month = municipality.findAll('option', {'selected': ''})[1]['value']
        temp_url = f"https://climate.weather.gc.ca/climate_data/daily_data_e.html?timeframe=2&Year={search_year}&Month={search_month}&Day={day_of_month}&hlyRange={hlyRange}&dlyRange={dlyRange}&mlyRange={mlyRange}&StationID={station_id}&Prov={prov}&urlExtension=_e.html&searchType=stnProv&optLimit=yearRange&StartYear=1840&EndYear={year}&selRowPerPage=100&Line=0&lstProvince="
        if mun_name:
            municipal_urls[mun_name] = {'url': temp_url, 'id': station_id, 'prov': prov, 'year': search_year, 'month': search_month}

In [7]:
base_url = f"https://climate.weather.gc.ca/historical_data/search_historic_data_stations_e.html?searchType=stnProv&timeframe=1&lstProvince=&optLimit=yearRange&StartYear=1840&EndYear=2022&Year=2022&Month=7&Day=4&selRowPerPage=100&txtCentralLatMin=0&txtCentralLatSec=0&txtCentralLongMin=0&txtCentralLongSec=0&startRow="
page = 1
while not is_last_page(soup, page):
    print(page)
    get_page_links(soup)
    next_url = base_url + str((page) * 100 + 1)
    r = requests.get(next_url)
    soup = BeautifulSoup(r.content)
    page += 1
get_page_links(soup)
print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=10700&Year=2007&Month=11&Day=1&time=&timeframe=2&submit=Download+Data

In [24]:
municipal_urls['(AE) BOW SUMMIT']

{'url': 'https://climate.weather.gc.ca/climate_data/daily_data_e.html?timeframe=2&Year=2&Month=3&Day=6&hlyRange=|&dlyRange=1998-02-01|2007-11-30&mlyRange=1998-04-01|2007-11-01&StationID=10700&Prov=AB&urlExtension=_e.html&searchType=stnProv&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=100&Line=0&lstProvince=',
 'id': '10700',
 'prov': 'AB',
 'year': '2',
 'month': '3'}

In [14]:
list_of_data = []
for name, info in municipal_urls.items():
    if not len(chosen_provinces):
        df = pd.read_csv(f"https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={info['id']}&Year={info['year']}&Month={info['month']}&Day=1&time=&timeframe=2&submit=Download+Data")
    else:
        if info['prov'] in chosen_provinces:
            df = pd.read_csv(f"https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={info['id']}&Year={info['year']}&Month={info['month']}&Day=1&time=&timeframe=2&submit=Download+Data")
            list_of_data.append(df)        

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [18]:
list_of_data

[]

In [13]:
for key, ValueError in municipal_urls.items():
    print(mun)

(AE) BOW SUMMIT
100 MILE HOUSE
100 MILE HOUSE 6NE
108 MILE HOUSE
108 MILE HOUSE ABEL LAKE
108 MILE HOUSE WATSON LAKE
150 MILE HOUSE
150 MILE HOUSE 7N
26 MILE DEPOT
70 MILE HOUSE
ABBEY
ABBOTSFORD
ABBOTSFORD A
ABBOTSFORD EAST
ABBOTSFORD KING ROAD
ABBOTSFORD UPPER SUMAS
ABEE AGDM
ABERCORN
ABERCROMBIE POINT
ABERDEEN
ABERFELDIE
ABITIBI CAMP 11
ABITIBI CAMP 228
ABITIBI CAMP 230
ABITIBI CAMP 300
ABITIBI CANYON
ABITIBI POST
ACADIA FOREST EXP ST
ACADIA VALLEY
ACADIA VALLEY CDA EPF
ACADIA VALLEY EXP ST
ACADIA VALLEY MACTAVISH
ACADIA VALLEY VANDYNE
ACADIEVILLE
ACME CDA EPF
ACTIVE PASS
ACTON BEARDMORE
ACTON MONITOR STN
ACTON VALE
ADAIR LO
ADAMS CREEK LO
ADAMS LAKE SKWAAM BAY
ADDENBROKE ISLAND
ADEN
ADRA
AGASSIZ CDA
AGASSIZ MOUNTAIN INSTITUTE
AGASSIZ RCS
AGNEW MINE
AGUASABON
AIGREMONT
AILSA CRAIG
AIRDRIE
AISHIHIK A
AIYANSH
AIYANSH 2SE
AKAMINA PASS
AKLAVIK A
AKLAVIK RADIOSONDE
AKULIVIK A
ALAMEDA CDA EPF
ALBANEL
ALBANY
ALBERNI BEAVER CREEK
ALBERNI CITY RESERVOIR
ALBERNI LUPSI CUPSI
ALBERNI MCCOY LAKE


In [ ]:
final_df = pd.concat(list_of_data)